# Objective

This notebook is an exploratory data analysis on finding duplicates in verified foods. Since, it started as an exploration, it might be difficult to follow the logic but important pieces that can be reused would be the cosine similarity computation which does a pairwise similarity check for a column of here, verified food brand+description pairs. This was fairly fast due to the way it was implemented. 

The notebook mainly focuses on exact duplicate comparison within verified foods set. final_foods_to_delete contains food ids that need to be deleted. 



In [ ]:
# -*- coding:utf-8 -*-

import pandas as pd

import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import json

import csv



In [2]:
import snowflake.connector

# Connectio string
conn = snowflake.connector.connect(
                user='fillthisin@myfitnesspal.com',
                password='B13!',
#                 password='8xkE32SWttg95SzV',
                account='xba11282.us-east-1',
                warehouse='PA_REPORTING_WH',
#                 database='DEMO_DB',
#                 schema='public'
                )

cur = conn.cursor()

cur.execute("select * from ANALYSIS.PUBLIC.FOODS where verified = 1 and deleted = FALSE and IS_PUBLIC = True;")


KeyboardInterrupt: 

In [210]:
# rows = cur.fetchall()


In [211]:
# len(rows)

In [ ]:
# this needed to be run only once 
# df = pd.DataFrame(np.array(rows).reshape(473321,25), columns = ['ID','USER_ID','DESCRIPTION','FOOD_TYPE','RANK','DELETED','CREATED_AT','UPDATED_AT','BRAND','IS_PUBLIC','COUNTRY_CODE','BRAND_AS_URL','VERSION','ORIGINAL_FOOD_ID','BRAND_NAME_ID','EDIT_NOTES','FOOD_PROVIDER_ID','COUNTRY_ID','PROMOTED_FROM_USER_ID','PROMOTED_FROM_FOOD_ID','CONFIDENCE_LEVEL','VERIFIED','EXTERNAL_ID','EXTERNAL_VERSION','USDA_IMPORT_FLAG'])
# df.to_csv("res_from_snof.csv")

# df


df.columns

In [3]:
# read already saved resutls from csv 
df = pd.read_csv("res_from_snof.csv")
del df['Unnamed: 0']
df

,ID,USER_ID,DESCRIPTION,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,...,EDIT_NOTES,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG
0,153189791,0,Pasta Penne Regina Whole Wheat,0,0,False,2014-10-21 14:13:30,2014-10-21 14:13:30,Parkhurst,True,...,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0
1,153267850,0,Mixed Greens With Mesclun,0,0,False,2014-10-21 22:09:50,2014-10-21 22:09:50,Sodexo Healthcare,True,...,NaN,NaN,NaN,NaN,NaN,0,1,8.935857e+13,6.722247e+13,0
2,155053748,0,Potatoes French Fries Fresh Cut (Bostwick),0,0,False,2014-11-03 15:01:41,2014-11-03 15:01:41,Parkhurst,True,...,New recipe weight,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0
3,155061116,0,Packaged Salads - Bacon Lettuce Toss Kit,0,19,False,2014-11-03 15:52:57,2014-11-03 15:52:57,Dole,True,...,New recipe weight,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0
4,155098908,0,Jacket Potato with Bean Chilli,0,0,False,2014-11-03 19:14:28,2014-11-03 19:14:28,KYF,True,...,New recipe weight,NaN,NaN,NaN,NaN,0,1,2.042510e+14,3.190281e+13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473316,1421477663,3,Organic Extra Virgin Olive Oil,0,0,False,2021-04-08 21:10:31,2021-04-08 21:10:31,Kirkland,True,...,patched serving sizes from recipe service,0.0,254.0,0.0,154998038.0,0,1,9.855869e+13,1.984873e+14,0
473317,1421509277,0,Fresh Tomato Salsa,3,0,False,2021-04-08 21:22:34,2021-04-08 21:22:34,Chipotle,True,...,NaN,NaN,254.0,NaN,NaN,0,1,1.984959e+14,1.984959e+14,0
473318,1421519103,0,Classic with Chicken Salad,3,0,False,2021-04-08 21:38:51,2021-04-08 21:38:51,Panera Bread,True,...,NaN,NaN,254.0,NaN,NaN,0,1,2.688644e+14,2.688644e+14,0
473319,1421558882,3,Butter,0,0,False,2021-04-08 22:22:12,2021-04-08 22:22:12,1 tsp,True,...,patched serving sizes from recipe service,0.0,0.0,0.0,38833659.0,0,1,1.334594e+14,2.683041e+14,0


In [4]:
# lower everything 
df['DESCRIPTION'] = df['DESCRIPTION'].str.lower()
df['BRAND'] = df['BRAND'].str.lower()


In [5]:
# algorithmically added foods are 17% of total 
# initially removed them but added them back because some of the most frequently logged VFs are of this type
df[df['USER_ID']==3].shape
df[df['USER_ID']==3]


,ID,USER_ID,DESCRIPTION,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,...,EDIT_NOTES,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG
36,194584115,3,chicken pot pie,0,0,False,2015-05-08 23:24:55,2015-05-08 23:24:55,nutrisystem,True,...,serving sizes requests.patch_food,0.0,254.0,0.0,153142032.0,0,1,9.855209e+13,1.952471e+13,0
37,194584318,3,mashed sweet potatoes,0,0,False,2015-05-08 23:25:44,2015-05-08 23:25:44,bob evans,True,...,serving sizes requests.patch_food,0.0,0.0,0.0,53680248.0,0,1,NaN,NaN,0
38,194585292,3,cheese puffs,0,0,False,2015-05-08 23:29:59,2015-05-08 23:29:59,smart puffs,True,...,serving sizes requests.patch_food,0.0,254.0,0.0,153402277.0,0,1,9.800029e+13,5.415729e+13,0
39,194585996,3,shanghai style beef,0,0,False,2015-05-08 23:33:16,2015-05-08 23:33:16,pf chang's,True,...,serving sizes requests.patch_food,0.0,0.0,0.0,49037108.0,0,1,6.281580e+13,1.245261e+14,0
40,194590183,3,"quarter chicken, white meat skinless",0,0,False,2015-05-08 23:52:17,2015-05-08 23:52:17,swiss chalet,True,...,serving sizes requests.patch_food,0.0,43.0,0.0,118073349.0,0,1,1.337364e+14,8.989141e+13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473312,1477102570,3,organic nutritional shake,0,0,False,2021-05-25 03:42:48,2021-05-25 03:42:48,organ,True,...,patched serving sizes from recipe service,0.0,254.0,0.0,79727364.0,0,1,6.336783e+13,6.174204e+13,0
473313,1477167951,3,3/4 cup,0,0,False,2021-05-25 07:15:24,2021-05-25 07:15:24,cheerios,True,...,patched serving sizes from recipe service,0.0,0.0,0.0,51037367.0,0,1,2.846465e+13,6.229181e+13,0
473315,1421416493,3,white choc,0,0,False,2021-04-08 19:47:54,2021-04-08 19:47:54,options,True,...,patched serving sizes from recipe service,0.0,0.0,0.0,42555887.0,0,1,2.791490e+13,2.339442e+14,0
473316,1421477663,3,organic extra virgin olive oil,0,0,False,2021-04-08 21:10:31,2021-04-08 21:10:31,kirkland,True,...,patched serving sizes from recipe service,0.0,254.0,0.0,154998038.0,0,1,9.855869e+13,1.984873e+14,0


In [6]:
# This time we want to only look at food type 0 which are the only ones showing up in search
# 176840 is 37% of whole rows/food ids are of type 3
df[df['FOOD_TYPE']==3].shape
df = df[df['FOOD_TYPE']==0]
df.shape
#62% of data food type 0

(296481, 25)

In [7]:
df.shape

(296481, 25)

In [8]:
# remove verified foods with external_id as null 

# df = df[df['EXTERNAL_ID'].notna()]

In [9]:
# unique pairs of desc and brand
df.groupby(['DESCRIPTION', 'BRAND']).ngroups

279650

In [10]:
# 473252 no duplicate food id
df['ORIGINAL_FOOD_ID'].nunique()
#473252
# df.shape

296481

In [12]:
# number of desc,brand pairs with 2 or more duplicates
DB_counts_df = df.groupby(['DESCRIPTION', 'BRAND'],as_index=False).size()
DB_counts_df[DB_counts_df['size']>1]



,DESCRIPTION,BRAND,size
764,1% lowfat milk,meijer,2
1474,100% whole wheat,arnold,2
1479,100% whole wheat,brownberry,2
1519,100% whole wheat bread,brownberry,2
2254,2% milk,horizon,2
...,...,...,...
256788,яблоко,m&s,3
274243,ミックスサンド,m&s,2
276472,水果,m&s,2
277519,米饭,m&s,2


In [13]:
DB_counts_df['size'].sum()


280437

In [17]:
# Group by food type:

# 0 foods - search
# 3 restaurants
df.groupby('FOOD_TYPE').count()

,ID,USER_ID,DESCRIPTION,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,COUNTRY_CODE,...,EDIT_NOTES,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG
FOOD_TYPE,,,,,,,,,,,,,,,,,,,,,
0,296481,296481,296481,296481,296481,296167,296481,280437,296481,201776,...,61978,84988,122679,84988,84988,296481,296481,144484,144484,296481


# Cosine similarity 

In [19]:
def ngrams(string, n=3):
    # fix encoding
#     string = fix_text(string)
    #remove non ascii chats
    string = string.encode("ascii",errors = "ignore").decode()
    string = string.lower()
    chars_to_remove = [")","(","{","}","'","[","]",".",","]
    rx = '['+re.escape(''.join(chars_to_remove))+']'
#     string = re.sub(rx,)
    string = ' '+string+' '
    string = re.sub(r'[,.-/]|\s',r'',string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]
    

In [28]:
df = df[df['DESCRIPTION'].notna()]

df[df['DESCRIPTION'].isna()]

,ID,USER_ID,DESCRIPTION,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,...,EDIT_NOTES,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG


In [29]:
# only using description 
from sklearn.feature_extraction.text import TfidfVectorizer

food_names = df['DESCRIPTION'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(food_names)

In [31]:
import numpy as np
from scipy.sparse import csr_matrix
# !pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct
def cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    print (A.shape)
    M,_ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
import time
t1 = time.time()
matches = cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)

In [ ]:
# create a df out of the sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})
        

In [ ]:
matches_df = get_matches_df(matches, food_names, top=100000)
matches_df = matches_df[matches_df['similarity'] < 2] # Remove all exact matches


In [ ]:
top10k = matches_df.sort_values(by=['similarity'], ascending = False)

In [ ]:
top10k.to_csv("top_10k_similar.csv")


# Combining description and brand


In [20]:
# 0 foods - search
# 3 restaurants
df.groupby('FOOD_TYPE').count()

,ID,USER_ID,DESCRIPTION,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,COUNTRY_CODE,...,EDIT_NOTES,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG
FOOD_TYPE,,,,,,,,,,,,,,,,,,,,,
0,296481,296481,296481,296481,296481,296167,296481,280437,296481,201776,...,61978,84988,122679,84988,84988,296481,296481,144484,144484,296481


In [21]:
# 123402 unique desc_brand from restaurants out of 176840 total rows from restaurants
# df[df['FOOD_TYPE']==3].groupby(['DESCRIPTION','BRAND']).count()

# 279673 unique desc_brand from foods out of 296481 rows from foods
# df[df['FOOD_TYPE']==0].groupby(['DESCRIPTION','BRAND']).count()
# df[df['FOOD_TYPE']==0].shape

In [23]:
cols = ['DESCRIPTION', 'BRAND']
df['desc_brand'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

<ipython-input-23-dbd5487ff247>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['desc_brand'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [1]:
food_br_names = df['desc_brand'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_fd_br = vectorizer.fit_transform(food_br_names)

In [2]:
# run cosine sim
t1 = time.time()
matches = cossim_top(tf_idf_matrix_fd_br, tf_idf_matrix_fd_br.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)

In [342]:
matches_df = get_matches_df(matches, food_br_names, top=200000)
matches_df = matches_df[matches_df['similarity'] < 0.99] # Remove all exact matches


In [343]:
df_non_dup = df.loc[~df.desc_brand.duplicated(keep='first')]
df_non_dup.shape
# df.shape
matches_df['left_food_type'] = matches_df['left_side'].map(df_non_dup.set_index('desc_brand')['FOOD_TYPE'])
matches_df['right_food_type'] = matches_df['right_side'].map(df_non_dup.set_index('desc_brand')['FOOD_TYPE'])

matches_df

,left_side,right_side,similarity,left_food_type,right_food_type
2,Mixed Greens With Mesclun Sodexo Healthcare,"Mixed Greens with Mesclun Sodexo (Campus, Heal...",0.867405,0,0
3,Mixed Greens With Mesclun Sodexo Healthcare,"Mixed Greens with Mesclun Sodexo (Healthcare, ...",0.865225,0,0
12,Sliced Pastrami OKSTATE Bread Beyond Deli,Sliced Salami OKSTATE Bread Beyond Deli,0.874125,0,0
17,100% Irish Minced Beef Meatballs in a Tangy To...,Minced Beef Meatballs in a Tangy Tomato Sauce ...,0.914264,0,0
24,Crispy Chinese Vegetable Spring Rolls KYF ROI,Crispy Chinese Vegetable Spring Rolls KYF NI,0.912160,0,0
...,...,...,...,...,...
199986,"Whipped Sweet Potato Sodexo (Healthcare, Campus)","Whipped Sweet Potatoes Sodexo (Healthcare, Cam...",0.912425,0,0
199991,Sauteed Spinach in Olive Oil with Garlic Sodex...,Sauteed Spinach in Olive Oil with Garlic Sodex...,0.965575,0,0
199992,Sauteed Spinach in Olive Oil with Garlic Sodex...,Sauteed Spinach in Olive Oil with Garlic Sodex...,0.912720,0,0
199993,Sauteed Spinach in Olive Oil with Garlic Sodex...,Sauteed Spinach in Olive Oil with Garlic Sodex...,0.905523,0,0


In [ ]:
# trying with different thresholds


In [350]:
matches_df2 = matches_df[matches_df['similarity'] >= 0.99]
matches_df2

,left_side,right_side,similarity,left_food_type,right_food_type
1569,Steak Burrito Bowl Freebirds World Burrito,Steak Burrito Bowl Freebirds World Burritos,0.991589,3,3
1925,Chicken Burrito Bowl Freebirds World Burrito,Chicken Burrito Bowl Freebirds World Burritos,0.991274,3,3
4196,Freebird Chicken Freebirds World Burritos,Freebird Chicken Freebirds World Burrito,0.992180,3,3
4898,Shaved Brussels Sprouts & Roasted Beet^ Cooper...,Shaved Brussels Sprouts & Roasted Beet^ Cooper...,0.991287,3,3
6918,Créme Brulee Perry's Steakhouse & Grill,Créme Brulee Perry's Steakhouse & Grille,0.992223,3,3
11884,Steak Burrito Bowl Freebirds World Burritos,Steak Burrito Bowl Freebirds World Burrito,0.991589,3,3
12777,Avocado Super Duper Burger,Avocado Super Duper Burgers,0.990875,3,3
22182,Roast Turkey And Brie Sandwich Grand Lux Cafe,Roast Turkey and Brie Sandwich Grand Lux Café,0.990399,3,3
39115,Jumbo Lump Crab Cakes Perry's Steakhouse & Grill,Jumbo Lump Crab Cakes Perry's Steakhouse & Grille,0.993628,3,3
39348,Jumbo Lump Crab Cakes Perry's Steakhouse & Grille,Jumbo Lump Crab Cakes Perry's Steakhouse & Grill,0.993628,3,3


In [352]:
matches_df3 = matches_df[(matches_df['similarity'] >= 0.95) &(matches_df['similarity'] < 0.99)]
matches_df3

,left_side,right_side,similarity,left_food_type,right_food_type
57,"Steamed Hot Dog with Jalapeo Chillies, Sour C...","Grilled Hot Dog with Jalapeo Chillies, Sour C...",0.961519,0,0
58,"Steamed Hot Dog with Jalapeo Chillies, Sour C...","Steamed Hot Dog with Jalapeo Chillies, Sour C...",0.959271,0,0
69,Cheese Puffs Smart puffs,Baked Cheese Puffs Smart Puffs,0.956424,0,0
582,Toffee Mocha Light Frappuccino® Blended Coffee...,Toffee Mocha Light Frappuccino® Blended Coffee...,0.958134,3,3
670,4oz Low Fat Cottage Cheese Continental Café,8oz Low Fat Cottage Cheese Continental Café,0.956175,0,0
...,...,...,...,...,...
199837,Lavender White Mocha (16 oz) Sodexo (Healthcar...,Iced Lavender White Mocha (16 oz) Sodexo (Heal...,0.956163,0,0
199843,Iced Lavender White Mocha (16 oz) Sodexo (Heal...,Lavender White Mocha (16 oz) Sodexo (Healthcar...,0.956163,0,0
199849,"Lavender White Mocha (16 oz) Sodexo (Campus, H...",Iced Lavender White Mocha (16 oz) Sodexo (Camp...,0.955987,0,0
199856,Iced Lavender White Mocha (16 oz) Sodexo (Camp...,"Lavender White Mocha (16 oz) Sodexo (Campus, H...",0.955987,0,0


In [353]:
matches_df4 = matches_df[(matches_df['similarity'] >= 0.90) &(matches_df['similarity'] < 0.95)]
matches_df4

,left_side,right_side,similarity,left_food_type,right_food_type
17,100% Irish Minced Beef Meatballs in a Tangy To...,Minced Beef Meatballs in a Tangy Tomato Sauce ...,0.914264,0,0
24,Crispy Chinese Vegetable Spring Rolls KYF ROI,Crispy Chinese Vegetable Spring Rolls KYF NI,0.912160,0,0
59,"Steamed Hot Dog with Jalapeo Chillies, Sour C...","Grilled Hot Dog with Jalapeo Chillies, Sour C...",0.920442,0,0
65,Chicken Pot Pie Nutrisystem,Pot Pie Nutrisystem,0.906715,0,0
79,Roasted Turkey Breast Safeway,Oven Roasted Turkey Breast Safeway,0.907795,0,0
...,...,...,...,...,...
199893,Korean BBQ Chicken Cheddar & Onion Sodexo (Cam...,"Korean BBQ Chicken, Cheddar & Onion Flat Sodex...",0.908808,0,0
199981,Asian Velvet Corn Soup Sodexo Healthcare,Velvet Corn Soup Sodexo Healthcare,0.900015,0,0
199986,"Whipped Sweet Potato Sodexo (Healthcare, Campus)","Whipped Sweet Potatoes Sodexo (Healthcare, Cam...",0.912425,0,0
199992,Sauteed Spinach in Olive Oil with Garlic Sodex...,Sauteed Spinach in Olive Oil with Garlic Sodex...,0.912720,0,0


In [354]:
matches_df5 = matches_df[(matches_df['similarity'] >= 0.85) &(matches_df['similarity'] < 0.90)]
matches_df5

,left_side,right_side,similarity,left_food_type,right_food_type
2,Mixed Greens With Mesclun Sodexo Healthcare,"Mixed Greens with Mesclun Sodexo (Campus, Heal...",0.867405,0,0
3,Mixed Greens With Mesclun Sodexo Healthcare,"Mixed Greens with Mesclun Sodexo (Healthcare, ...",0.865225,0,0
12,Sliced Pastrami OKSTATE Bread Beyond Deli,Sliced Salami OKSTATE Bread Beyond Deli,0.874125,0,0
27,Chickpea Salad Sandwich No Gluten ARAMARK Cana...,Tuna Salad Sandwich No Gluten ARAMARK Canada E...,0.899643,0,0
32,Creamy Mushroom Sauce KYF,Pork in a Creamy Mushroom Sauce KYF,0.873300,0,0
...,...,...,...,...,...
199958,"Savory Oatmeal and Egg Bowl Sodexo (Campus, He...",Savory Oatmeal and Egg Bowl Sodexo Campus,0.860992,0,0
199976,Grilled Carrots Sodexo Healthcare,Roasted Carrots Sodexo Healthcare,0.863238,0,0
199977,Grilled Carrots Sodexo Healthcare,Sliced Carrots Sodexo Healthcare,0.857985,0,0
199983,Chicken Teriyaki Stir-Fry Sodexo Healthcare,"Chicken Teriyaki Stir-Fry Sodexo (Campus, Heal...",0.869353,0,0


In [362]:
# repeat the analysis for matches_df with lower threshold of 0.5 



# run cosine sim
t1 = time.time()
matches = cossim_top(tf_idf_matrix_fd_br, tf_idf_matrix_fd_br.transpose(), 10, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)


(418062, 31431)
SELFTIMED: 4786.803067922592


In [344]:
#43787
# matches_df
print(matches_df[matches_df['left_food_type']==3].shape[0]/(matches_df[matches_df['left_food_type']==0].shape[0]+matches_df[matches_df['left_food_type']==3].shape[0])*100)
print(matches_df[matches_df['right_food_type']==3].shape[0]/(matches_df[matches_df['right_food_type']==0].shape[0]+matches_df[matches_df['left_food_type']==3].shape[0])*100)


11.518515980170426
10.022989542012262


# Comparison with Top Logged Foods

In [ ]:

# import snowflake.connector

# # Connectio string
# conn = snowflake.connector.connect(
#                 user='fillthisin@myfitnesspal.com',
#                 password='B123!',
# #                 password='8xkE32SWttg95SzV',
#                 account='xba11282.us-east-1',
#                 warehouse='ADHOC_SEARCH',
#                 database='ANALYSIS',
#                 schema='SEARCH'
#                 )

# # Create cursor
# cur = conn.cursor()

# # Execute SQL statement
# cur.execute("select QUERY from tmp_query_based_log_counts_dataset_20210101_to_20210531 where QUERY_RANK <= 5000 order by QUERY_RANK asc ;")
# # rows = cursor.fetchall()

# # Fetch result
# # print (cur.fetchall())

In [ ]:
# run once

rows = cur.fetchall()





In [ ]:
len(rows)

In [ ]:
# run once
top_logged_queries = pd.DataFrame(np.array(rows).reshape(4987,1), columns = ['query'])
top_logged_queries.to_csv("top_logged_queries.csv")
top_logged_queries

In [ ]:
top_logged_queries = pd.read_csv("top_logged_queries.csv")
top_logged_queries

In [ ]:
# top_logged_queries.shape
# len(food_names)
tf_idf_matrix

In [ ]:
# # tf id for the food brand
# from sklearn.feature_extraction.text import TfidfVectorizer

# food_br_names = df['desc_brand'].unique()
# vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
# tf_idf_matrix_fd_br = vectorizer.fit_transform(food_br_names)
# tf_idf_matrix_fd_br

In [ ]:
# tf id for the top queries
topl = top_logged_queries['query']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_query = vectorizer.fit_transform(topl)
tf_idf_matrix_query

In [ ]:
# run cosine sim
t1 = time.time()
matches = cossim_top(tf_idf_matrix_query, tf_idf_matrix.transpose(), 10, 0.6)
t = time.time()-t1
print("SELFTIMED:", t)

In [ ]:
len(food_names)
food_names

In [ ]:
topl

In [ ]:
# create a df out of the sparse matrix
def get_matches_df2(sparse_matrix, name_vector1,name_vector2, top=100):
    non_zeros = sparse_matrix.nonzero()
    print (name_vector2[non_zeros[0][0]])
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    print (top)
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        print (sparserows[index], "*",name_vector1[sparserows[index]])
        print (sparsecols[index],"**",name_vector2[sparsecols[index]])
        left_side[index] = name_vector1[sparserows[index]]
        right_side[index] = name_vector2[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})
        

In [ ]:
q_matches_df = get_matches_df2(matches,topl,food_names, top=10)
q_matches_df = matches_df[matches_df['similarity'] < 0.99] # Remove all exact matches
q_matches_df

In [ ]:
top_logged_queries['query']

In [ ]:
# left side description of the verified food
df[df['DESCRIPTION']=='Scandinavian Mixed Vegetables']

# right side top logged query
# top_logged_queries[top_logged_queries['query'].str.contains('Scandinavian Mixed Vegetable')]
# q_matches_df[q_matches_df['right_side'] == 'Mixed Greens with Mesclun']

In [ ]:
q_matches_df.sort_values(by=['similarity'], ascending = False)

In [ ]:
q_matches_df.sort_values(by=['similarity'], ascending = False).to_csv("verified_logged_sim1.csv")

In [ ]:
[len(set(a).intersection(b)) for a, b in zip(top_logged_queries.query, df.DESCRIPTION)]


# log count for verified foods

In [420]:
# Only getting data for food ids, and not grouped by version
# for each food id of duplicate dec,brand, we want to check how many times they are logged 
import snowflake.connector
conn = snowflake.connector.connect(
                user='fillthisin@myfitnesspal.com',
                password='B13!!',
#                 password='8xkE32SWttg95SzV',
                account='xba11282.us-east-1',
                warehouse='PA_REPORTING_WH',
                database='ANALYSIS',
                schema='PUBLIC'
                )

# Create cursor
cur = conn.cursor()

# Execute SQL statement
cur.execute("select ORIG_FOOD_ID,DESCRIPTION,BRAND,COUNT(*) as log_count from  VERIFIED_FOOD_ENTRIES group by ORIG_FOOD_ID,DESCRIPTION,BRAND order by log_count desc")

# rows = cursor.fetchall()

# Fetch result
# print (cur.fetchall())





In [421]:
rows = cur.fetchall()
len(rows)

245146

In [422]:
# write only once"

# vf_logged_no_fd_version_df = pd.DataFrame(np.array(rows).reshape(245146,4), columns = ['OBF_ORIG_FOOD_ID','DESCRIPTION','BRAND','LOG_COUNT'])
# vf_logged_no_fd_version_df.to_csv("vf_logged_no_fd_version_df.csv")


# vf_logged_no_fd_version_df = pd.DataFrame(np.array(rows).reshape(245146,4), columns = ['ORIG_FOOD_ID','DESCRIPTION','BRAND','LOG_COUNT'])
# vf_logged_no_fd_version_df.to_csv("vf_logged_no_fd_version_df_ids.csv")
vf_logged_no_fd_version_df

,ORIG_FOOD_ID,DESCRIPTION,BRAND,LOG_COUNT
0,709641699,Banana,None,48141523
1,709641731,Egg,None,28969287
2,709641696,Avocado,None,16509053
3,709641727,Black coffee,None,14072726
4,709641705,Blueberries,None,13592189
...,...,...,...,...
245141,1393655812,chorizo mushroom scramble with tortilla (63009.1),Eurest,1
245142,514093321,Dal Rajma - Kidney Beans with Ginger and Tomato,Sodexo Workplace,1
245143,194274183,Homemade Bakewell Tart,ESS,1
245144,716904964,Roasted Chicken Farro Eggplant (79972.1),Flik,1


In [27]:
# vf_logged_no_fd_version_df = pd.read_csv("vf_logged_no_fd_version_df.csv")
vf_logged_no_fd_version_df = pd.read_csv("vf_logged_no_fd_version_df_ids.csv")

del vf_logged_no_fd_version_df['Unnamed: 0']
vf_logged_no_fd_version_df



,ORIG_FOOD_ID,DESCRIPTION,BRAND,LOG_COUNT
0,709641699,Banana,NaN,48141523
1,709641731,Egg,NaN,28969287
2,709641696,Avocado,NaN,16509053
3,709641727,Black coffee,NaN,14072726
4,709641705,Blueberries,NaN,13592189
...,...,...,...,...
245141,1393655812,chorizo mushroom scramble with tortilla (63009.1),Eurest,1
245142,514093321,Dal Rajma - Kidney Beans with Ginger and Tomato,Sodexo Workplace,1
245143,194274183,Homemade Bakewell Tart,ESS,1
245144,716904964,Roasted Chicken Farro Eggplant (79972.1),Flik,1


In [28]:
vf_logged_no_fd_version_df['DESCRIPTION'] = vf_logged_no_fd_version_df['DESCRIPTION'].str.lower()
vf_logged_no_fd_version_df['BRAND'] = vf_logged_no_fd_version_df['BRAND'].str.lower()


In [30]:
# on the other hand we have duplicate brand,desc combos all in DB_counts_df 
# but they do not have ids because they are aggregated 
# retrieve all with log count greater than one in s
s = DB_counts_df[DB_counts_df['size']>1]
s.head()

,DESCRIPTION,BRAND,size
764,1% lowfat milk,meijer,2
1474,100% whole wheat,arnold,2
1479,100% whole wheat,brownberry,2
1519,100% whole wheat bread,brownberry,2
2254,2% milk,horizon,2


In [31]:
# the number of duplicates in each group 
DB_counts_df[DB_counts_df['size']>1].sort_values(by = 'size',ascending = False)

,DESCRIPTION,BRAND,size
80384,dark chocolate,mars,194
256534,кефир 1%,m&s,11
256698,творог,m&s,11
256733,творог обезжиренный,m&s,10
256512,йогурт,m&s,8
...,...,...,...
84705,egg,egg,2
84556,edamame,trader joe's,2
84526,edamame,birds eye,2
83273,double quarter pounder with cheese,mcdonald's,2


In [35]:
# find all the food ids which have desc,brand equal to the desc,brands in set s which is all desc,brand combos
# which have more than 1 duplicate
 
    
exact_dups_with_ids = pd.merge(s, df,  how='inner', left_on=['DESCRIPTION','BRAND'], right_on = ['DESCRIPTION','BRAND'])
exact_dups_with_ids



,DESCRIPTION,BRAND,size,ID,USER_ID,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,...,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG,desc_brand
0,1% lowfat milk,meijer,2,914747337,3,0,0,False,2019-05-29 18:19:12,2019-05-29 18:19:12,...,0.0,254.0,0.0,78781683.0,0,1,9.855865e+13,1.945078e+14,0,1% lowfat milk meijer
1,1% lowfat milk,meijer,2,388356908,3,0,0,False,2016-11-16 15:24:22,2016-11-16 15:24:22,...,0.0,254.0,0.0,78781683.0,0,1,1.382977e+14,1.719507e+14,0,1% lowfat milk meijer
2,100% whole wheat,arnold,2,188882081,3,0,0,False,2015-04-15 03:58:10,2015-04-15 03:58:10,...,0.0,254.0,0.0,85228290.0,0,1,9.854996e+13,9.854996e+13,0,100% whole wheat arnold
3,100% whole wheat,arnold,2,188908861,3,0,0,False,2015-04-15 04:15:13,2015-04-15 04:15:13,...,0.0,254.0,0.0,142830949.0,0,1,NaN,NaN,0,100% whole wheat arnold
4,100% whole wheat,brownberry,2,188843954,3,0,0,False,2015-04-15 03:33:53,2015-04-15 03:33:53,...,0.0,0.0,0.0,60793788.0,0,1,NaN,NaN,0,100% whole wheat brownberry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,水果,m&s,2,188910505,3,0,0,False,2015-04-15 04:16:17,2015-04-15 04:16:17,...,0.0,49.0,0.0,145246641.0,0,1,NaN,NaN,0,水果 m&s
1305,米饭,m&s,2,218035915,3,0,0,False,2015-07-30 05:01:12,2015-07-30 05:01:12,...,0.0,49.0,0.0,123515673.0,0,1,NaN,NaN,0,米饭 m&s
1306,米饭,m&s,2,188899122,3,0,0,False,2015-04-15 04:09:01,2015-04-15 04:09:01,...,0.0,49.0,0.0,127877094.0,0,1,NaN,NaN,0,米饭 m&s
1307,香蕉,m&s,2,206526504,3,0,0,False,2015-06-19 02:30:17,2015-06-19 02:30:17,...,0.0,49.0,0.0,153758930.0,0,1,NaN,NaN,0,香蕉 m&s


In [36]:
# I dont care abt external id null because I am joining the exact_dups_with_ids with a subset of 
# FOOD_ENTRIES_JOIN_FOODS which basicaly has the same fields 

df[df['ORIGINAL_FOOD_ID'].notna()]

,ID,USER_ID,DESCRIPTION,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,BRAND,IS_PUBLIC,...,FOOD_PROVIDER_ID,COUNTRY_ID,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG,desc_brand
0,153189791,0,pasta penne regina whole wheat,0,0,False,2014-10-21 14:13:30,2014-10-21 14:13:30,parkhurst,True,...,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0,pasta penne regina whole wheat parkhurst
1,153267850,0,mixed greens with mesclun,0,0,False,2014-10-21 22:09:50,2014-10-21 22:09:50,sodexo healthcare,True,...,NaN,NaN,NaN,NaN,0,1,8.935857e+13,6.722247e+13,0,mixed greens with mesclun sodexo healthcare
2,155053748,0,potatoes french fries fresh cut (bostwick),0,0,False,2014-11-03 15:01:41,2014-11-03 15:01:41,parkhurst,True,...,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0,potatoes french fries fresh cut (bostwick) par...
3,155061116,0,packaged salads - bacon lettuce toss kit,0,19,False,2014-11-03 15:52:57,2014-11-03 15:52:57,dole,True,...,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0,packaged salads - bacon lettuce toss kit dole
4,155098908,0,jacket potato with bean chilli,0,0,False,2014-11-03 19:14:28,2014-11-03 19:14:28,kyf,True,...,NaN,NaN,NaN,NaN,0,1,2.042510e+14,3.190281e+13,0,jacket potato with bean chilli kyf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473313,1477167951,3,3/4 cup,0,0,False,2021-05-25 07:15:24,2021-05-25 07:15:24,cheerios,True,...,0.0,0.0,0.0,51037367.0,0,1,2.846465e+13,6.229181e+13,0,3/4 cup cheerios
473315,1421416493,3,white choc,0,0,False,2021-04-08 19:47:54,2021-04-08 19:47:54,options,True,...,0.0,0.0,0.0,42555887.0,0,1,2.791490e+13,2.339442e+14,0,white choc options
473316,1421477663,3,organic extra virgin olive oil,0,0,False,2021-04-08 21:10:31,2021-04-08 21:10:31,kirkland,True,...,0.0,254.0,0.0,154998038.0,0,1,9.855869e+13,1.984873e+14,0,organic extra virgin olive oil kirkland
473319,1421558882,3,butter,0,0,False,2021-04-08 22:22:12,2021-04-08 22:22:12,1 tsp,True,...,0.0,0.0,0.0,38833659.0,0,1,1.334594e+14,2.683041e+14,0,butter 1 tsp


In [63]:
exact_dups_with_ids.shape

(1309, 27)

In [97]:
# exact_dups_with_ids_no_null.shape
vf_logged_no_fd_version_df.columns

Index(['ORIG_FOOD_ID', 'DESCRIPTION', 'BRAND', 'LOG_COUNT'], dtype='object')

In [72]:
# join duplicate foods exact_dups_with_ids_no_null with vf_logged_no_fd_version_df which has log count
# the latter comes from foods_join_food_entries
# the first comes from foods table
# both hame similar fields so join on id 

# exact_dups_with_logcnt_no_fv = pd.merge(exact_dups_with_ids_no_null, vf_logged_no_fd_version_df,  how='left', left_on=['EXTERNAL_ID'], right_on = ['OBF_ORIG_FOOD_ID'])
# exact_dups_with_logcnt_no_fv.sort_values(by = 'LOG_COUNT',ascending = False)

exact_dups_with_logcnt_no_fv = pd.merge(exact_dups_with_ids, vf_logged_no_fd_version_df,  how='left', left_on=['ORIGINAL_FOOD_ID','BRAND','DESCRIPTION'], right_on = ['ORIG_FOOD_ID','BRAND','DESCRIPTION'])
exact_dups_with_logcnt_no_fv.sort_values(by = 'LOG_COUNT',ascending = False)



,DESCRIPTION,BRAND,size,ID,USER_ID,FOOD_TYPE,RANK,DELETED,CREATED_AT,UPDATED_AT,...,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG,desc_brand,ORIG_FOOD_ID,LOG_COUNT
533,egg,egg,2,924336450,0,0,912,False,2019-06-13 22:43:57,2019-06-13 22:43:57,...,NaN,NaN,0,1,1.340262e+14,2.729598e+14,0,egg egg,1123.0,6778764.0
39,avocado,avocado,2,869737738,3,0,0,False,2019-03-16 03:32:03,2019-03-16 03:32:03,...,0.0,99253511.0,0,1,6.309298e+13,1.231514e+14,0,avocado avocado,188835480.0,3822396.0
532,egg,egg,2,340685752,113599533,0,0,False,2016-07-01 04:05:20,2016-07-01 04:05:20,...,NaN,NaN,0,1,1.602771e+14,1.672886e+14,0,egg egg,76888035.0,2585017.0
537,eggs,nice eggs,2,1129127988,3,0,0,False,2020-05-26 18:02:36,2020-05-26 18:02:36,...,0.0,97191477.0,0,1,3.274473e+13,1.334615e+14,0,eggs nice eggs,175936343.0,1992339.0
38,avocado,avocado,2,1278206630,3,0,0,False,2020-11-30 20:52:51,2020-11-30 20:52:51,...,0.0,126410553.0,0,1,1.337452e+14,2.079773e+14,0,avocado avocado,188908887.0,573096.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,творог обезжиренный,m&s,10,188906558,3,0,0,False,2015-04-15 04:13:43,2015-04-15 04:13:43,...,0.0,140696867.0,0,1,NaN,NaN,0,творог обезжиренный m&s,NaN,NaN
1283,творог обезжиренный,m&s,10,188913577,3,0,0,False,2015-04-15 04:18:15,2015-04-15 04:18:15,...,0.0,112452241.0,0,1,NaN,NaN,0,творог обезжиренный m&s,NaN,NaN
1301,ミックスサンド,m&s,2,188904668,3,0,0,False,2015-04-15 04:12:32,2015-04-15 04:12:32,...,0.0,139217979.0,0,1,NaN,NaN,0,ミックスサンド m&s,NaN,NaN
1302,ミックスサンド,m&s,2,188906281,3,0,0,False,2015-04-15 04:13:33,2015-04-15 04:13:33,...,0.0,136553221.0,0,1,NaN,NaN,0,ミックスサンド m&s,NaN,NaN


# Final results for exact dups with food_type = 0

final_foods_logcount contains all food ids which have one or more dups exact. food type 0 

In [14]:
# we have once written final_foods_logcount, we just read it now. 

exact_dups_with_logcnt_no_fv = pd.read_csv("final_foods_logcount.csv")


In [15]:
col_list = list(exact_dups_with_logcnt_no_fv.columns)


# exact_dups_with_logcnt_no_fv.sort_values(by=['DESCRIPTION'],ascending = True).to_csv("final_foods_logcount.csv", columns = col_list,encoding='utf_8_sig')

In [5]:
exact_dups_with_logcnt_no_fv.sort_values(by=['DESCRIPTION'],ascending = True)

In [4]:
exact_dups_with_logcnt_no_fv[(exact_dups_with_logcnt_no_fv['DESCRIPTION']=='dark chocolate') & (exact_dups_with_logcnt_no_fv['BRAND']=='mars')]


In [6]:
# food ids which exist in duplicate list but do not have any log count in log count df
#screenshot this
# there are 194 

exact_dups_with_logcnt_no_fv[exact_dups_with_logcnt_no_fv.LOG_COUNT.isna()]

In [24]:
# we decided to bulk delete rows with user id 3 that have log count less than 10 
bulk_del_low_log_count_df = exact_dups_with_logcnt_no_fv[(exact_dups_with_logcnt_no_fv['LOG_COUNT']<=10) & (exact_dups_with_logcnt_no_fv['USER_ID']==3)]

In [27]:
bulk_del_low_log_count_df.sort_values(by=['DESCRIPTION'],ascending = True).to_csv("bulk_del_low_log_count.csv", columns = col_list,encoding='utf_8_sig')
bulk_del_low_log_count_df.shape

(146, 30)

In [31]:
# now remove these rows from the original result and decide for the rest

exact_dups_with_logcnt_no_fv[~((exact_dups_with_logcnt_no_fv['LOG_COUNT']<=10) & (exact_dups_with_logcnt_no_fv['USER_ID']==3))].sort_values(by=['DESCRIPTION'],ascending = True).to_csv("final_foods_logcount_after_bulk_delete.csv", columns = col_list,encoding='utf_8_sig')



In [49]:
exact_dups_after_bulk_del.head()

,Unnamed: 0,DESCRIPTION,BRAND,size,ID,USER_ID,FOOD_TYPE,RANK,DELETED,CREATED_AT,...,PROMOTED_FROM_USER_ID,PROMOTED_FROM_FOOD_ID,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG,desc_brand,ORIG_FOOD_ID,LOG_COUNT
0,0,1% lowfat milk,meijer,2,914747337,3,0,0,False,2019-05-29 18:19:12,...,0.0,78781683.0,0,1,9.855865e+13,1.945078e+14,0,1% lowfat milk meijer,188912875.0,93952.0
1,1,1% lowfat milk,meijer,2,388356908,3,0,0,False,2016-11-16 15:24:22,...,0.0,78781683.0,0,1,1.382977e+14,1.719507e+14,0,1% lowfat milk meijer,176000087.0,8036.0
2,2,100% whole wheat,arnold,2,188882081,3,0,0,False,2015-04-15 03:58:10,...,0.0,85228290.0,0,1,9.854996e+13,9.854996e+13,0,100% whole wheat arnold,188882081.0,24205.0
4,4,100% whole wheat,brownberry,2,188843954,3,0,0,False,2015-04-15 03:33:53,...,0.0,60793788.0,0,1,NaN,NaN,0,100% whole wheat brownberry,188843954.0,37.0
5,5,100% whole wheat,brownberry,2,188892874,3,0,0,False,2015-04-15 04:05:02,...,0.0,137051603.0,0,1,6.281592e+13,6.281592e+13,0,100% whole wheat brownberry,188892874.0,9305.0


In [124]:
exact_dups_after_bulk_del = exact_dups_with_logcnt_no_fv[~((exact_dups_with_logcnt_no_fv['LOG_COUNT']<=10) & (exact_dups_with_logcnt_no_fv['USER_ID']==3))]


In [7]:
# for ones with more than 1 dup, just choose the min manually because only a few of them exist
# t = exact_dups_after_bulk_del.groupby(['DESCRIPTION', 'BRAND'],as_index=False).size().sort_values(by='size',ascending = False)


In [125]:
# some of the desc brands only have one member - probably the dup has been deleted in the bulk delete phase
# first remove those 
# then from the ones which have dups, choose the ones with lower log count to delete
temp = exact_dups_after_bulk_del.groupby(['DESCRIPTION', 'BRAND'],as_index=False).size().sort_values(by='size',ascending = False)
temp = temp[temp['size']<2]
temp

,DESCRIPTION,BRAND,size
3,100% whole wheat bread,brownberry,1
462,whole wheat toast,brownberry,1
43,beef pot pie,schwan's,1
40,beef bourguignon,bistro md,1
37,bbq chips,schlotzsky's,1
...,...,...,...
114,chicken quesadilla,smart ones,1
109,chicken noodle soup,medifast,1
382,slim a soup golden vegetable,batchelors,1
383,slow roasted turkey breast,smart ones,1


In [146]:
df3 = pd.merge(exact_dups_after_bulk_del, temp, how='left', on = ['DESCRIPTION','BRAND'] , indicator=True)
df3


,Unnamed: 0,DESCRIPTION,BRAND,size_x,ID,USER_ID,FOOD_TYPE,RANK,DELETED,CREATED_AT,...,CONFIDENCE_LEVEL,VERIFIED,EXTERNAL_ID,EXTERNAL_VERSION,USDA_IMPORT_FLAG,desc_brand,ORIG_FOOD_ID,LOG_COUNT,size_y,_merge
0,0,1% lowfat milk,meijer,2,914747337,3,0,0,False,2019-05-29 18:19:12,...,0,1,9.855865e+13,1.945078e+14,0,1% lowfat milk meijer,188912875.0,93952.0,NaN,left_only
1,1,1% lowfat milk,meijer,2,388356908,3,0,0,False,2016-11-16 15:24:22,...,0,1,1.382977e+14,1.719507e+14,0,1% lowfat milk meijer,176000087.0,8036.0,NaN,left_only
2,2,100% whole wheat,arnold,2,188882081,3,0,0,False,2015-04-15 03:58:10,...,0,1,9.854996e+13,9.854996e+13,0,100% whole wheat arnold,188882081.0,24205.0,1.0,both
3,4,100% whole wheat,brownberry,2,188843954,3,0,0,False,2015-04-15 03:33:53,...,0,1,NaN,NaN,0,100% whole wheat brownberry,188843954.0,37.0,NaN,left_only
4,5,100% whole wheat,brownberry,2,188892874,3,0,0,False,2015-04-15 04:05:02,...,0,1,6.281592e+13,6.281592e+13,0,100% whole wheat brownberry,188892874.0,9305.0,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,1304,水果,m&s,2,188910505,3,0,0,False,2015-04-15 04:16:17,...,0,1,NaN,NaN,0,水果 m&s,188910505.0,155.0,NaN,left_only
1159,1305,米饭,m&s,2,218035915,3,0,0,False,2015-07-30 05:01:12,...,0,1,NaN,NaN,0,米饭 m&s,188913947.0,6006.0,NaN,left_only
1160,1306,米饭,m&s,2,188899122,3,0,0,False,2015-04-15 04:09:01,...,0,1,NaN,NaN,0,米饭 m&s,188899122.0,552.0,NaN,left_only
1161,1307,香蕉,m&s,2,206526504,3,0,0,False,2015-06-19 02:30:17,...,0,1,NaN,NaN,0,香蕉 m&s,188907775.0,9096.0,NaN,left_only


In [10]:
df3 = df3.loc[(df3['_merge'] == 'left_only')]
df3

In [154]:
df3 = df3.drop(columns=['size_y', '_merge'])
df3 = df3.sort_values('LOG_COUNT', ascending=True).drop_duplicates(['DESCRIPTION','BRAND']).sort_index()


In [155]:
df3.sort_values(by=['DESCRIPTION'],ascending = True).to_csv("final_foods_to_delete.csv", encoding='utf_8_sig')

In [142]:
df3 = df3.drop('size_y', 1)

In [152]:
df3.shape

(1065, 30)

In [ ]:
# note that, for duplicates with more than 2 dups, we would want to delete all 
# food items except for the one which has the highest log count. Therefore, the final list of foods to delete must contain 
# all dups with lower than max log count for that food. Some of these edits are done manually and the final file which contains
# the food ids to delete is : final_VF_foods_to_delete
# It has two tabs and all foods ids in both tabs need to be deleted.
